Ce script permet d'afficher tous les trains traversant la France ayant du retard (et uniquement du retard).

# Code

## Import des Modules

In [29]:
import pandas as pd
import requests
import json
import datetime

## Obtention des trains ayant eu des perturbations sur leur trajet

Mise en varaible du token de connexion à l'API SNCF V1

In [30]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

Requête à l'API SNCF afin de récupérer la liste des trains avec des perturbations d'aujourd'hui

L'API ne permettant que de récupérer 1000 résultats à la fois, on réalise une boucle afin de tout récupérer

In [31]:
# On récupère la date du jour ou on la saisit manuellement

date = datetime.date.today()
date = str(date)
date = date[:4] + date[5:7] + date[8:10]

#date = '20230307'

In [32]:
i = 0
val = True
df = pd.DataFrame()
while val:
    link = 'https://api.sncf.com/v1/coverage/sncf/disruptions?since=' + date +'T000000&start_page=' + str(i) + '&count=1000&'
    req = requests.get(link,auth=(token, ''))
    doc = json.loads(req.text)
    df = pd.concat([df, pd.DataFrame(doc['disruptions'])], ignore_index=True)
    i += 1
    if len(doc['disruptions']) != 1000:
        val = False

## Traitement des retards

On  ne garde que les informations sur les perturbations et le jour de ces dernières

In [33]:
df_jour = pd.DataFrame(list(df['application_periods']))
df_jour = pd.DataFrame(list(df_jour[0]))
df_retard = pd.DataFrame(list(df['severity']))
df_retard['updated_at'] = df_jour['begin']

On ne garde que les trains ayant eu du retard

In [34]:
df_retard = df_retard[df_retard['effect'] == 'SIGNIFICANT_DELAYS']
df_retard = df_retard[['effect','updated_at']]

On réalise une fonction de conversion d'heures afin de calculer le retard

In [35]:
def conv_heure(string):
    return int(string[0:2]) + int(string[2:4])/60

## Obtention d'inforamtions complémentaires sur ce retard

On ajoute des informations au retard, ligne par ligne :

- Les gares de départ et d'arrivée

- Le retard et sa cause

- Les heures d'arrivée prévues et réelles du train

- L'heure de départ prévue du train

In [36]:
gare_d = []
gare_a = []
cause = []
retard = []
heure_arr_prevue = []
heure_arr_reelle = []
heure_depart = []

#for i in df_retard.index:
for index, row in df_retard.iterrows():
    #df_mod = pd.DataFrame(list(df['impacted_objects'][i]))
    df_mod = pd.DataFrame(list(df.iloc[index]['impacted_objects']))
    df_mod = pd.DataFrame(list(df_mod['impacted_stops'])[0])
    df_mod = df_mod[(df_mod['amended_arrival_time'].notnull())]
    df_mod = df_mod[(df_mod['base_arrival_time'].notnull())]
    
    # On calcule le retard et on ajoute une exception si il y a un changement de jour entre les heures d'arrivée prévues et réelles
    retard_val = conv_heure(df_mod.iloc[-1]['amended_arrival_time']) - conv_heure(df_mod.iloc[-1]['base_arrival_time'])

    if retard_val<0:
        retard_val = int(round((retard_val+24)*60,1))
    else:
        retard_val = int(round(retard_val*60,1))
    
    df_mod['gare'] = pd.DataFrame(list(df_mod['stop_point']))['name']

    # On renvoie "Retard non expliqué" si la SNCF ne fournit pas plus d'informations sur le retard
    cause_ret = df_mod.iloc[-1]['cause']
    if cause_ret == "":
        cause_ret = "Retard non expliqué"

    # On ajoute les informations à des listes
    gare_d.append(df_mod.iloc[0]['gare'])
    gare_a.append(df_mod.iloc[-1]['gare'])
    cause.append(cause_ret)
    retard.append(retard_val)
    heure_arr_prevue.append(df_mod.iloc[-1]['base_arrival_time'])
    heure_arr_reelle.append(df_mod.iloc[-1]['amended_arrival_time'])
    heure_depart.append(df_mod.iloc[0]['base_departure_time'])


Création d'une fonction afin d'obtenir le jour du départ

In [37]:
def get_day(string):
    string = string[:8]
    return string[6:8]+'-'+string[4:6]+'-'+string[0:4]

On ajoute les informations au dataframe

In [38]:
df_retard = df_retard.drop(['effect'], axis=1)

df_retard['Gare (départ)'] = gare_d
df_retard['Gare (arrivée)'] = gare_a
df_retard['updated_at'] = df_retard['updated_at'].apply(get_day)
df_retard["Arrivée (réelle)"] = heure_arr_reelle
df_retard["Arrivée (prévue)"] = heure_arr_prevue
df_retard["Départ (prévu)"] = heure_depart
df_retard['Retard (min)'] = retard
df_retard['Cause'] = cause

df_retard.rename(columns = {'updated_at':'Jour'}, inplace = True)


On passe les heures au format heure de `datetime`

In [39]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_retard["Arrivée (réelle)"] = df_retard["Arrivée (réelle)"].apply(str_tps)
df_retard["Arrivée (prévue)"] = df_retard["Arrivée (prévue)"].apply(str_tps)
df_retard["Départ (prévu)"] = df_retard["Départ (prévu)"].apply(str_tps)

On affiche le dataframe final

In [40]:
df_retard

,Jour,Gare (départ),Gare (arrivée),Arrivée (réelle),Arrivée (prévue),Départ (prévu),Retard (min),Cause
73,09-03-2023,La Bresse - Place du Champtel,Remiremont,08:07:00,08:02:00,06:59:00,5,Embouteillage
75,09-03-2023,Frasne,Neuchâtel,21:58:00,21:53:00,20:53:00,5,Prise en charge de clients en correspondance
166,09-03-2023,Lyon Part Dieu,Paris - Gare de Lyon - Hall 1 & 2,13:39:00,13:34:00,11:34:00,5,Travaux sur les voies
210,09-03-2023,Sain-Bel,Lyon - Saint-Paul,15:51:00,15:46:00,15:04:00,5,Retard non expliqué
252,10-03-2023,Paris - Montparnasse - Hall 1 & 2,Quimper,12:43:00,12:38:00,08:40:00,5,Panne d'un aiguillage
...,...,...,...,...,...,...,...,...
21013,10-03-2023,Lyon Part Dieu,Annecy,10:12:00,10:07:00,08:08:00,5,Régulation du trafic
21014,10-03-2023,Poitiers,Limoges-Bénédictins,11:42:30,10:47:30,08:51:00,55,Réutilisation d'un train
21018,10-03-2023,Limoges-Bénédictins,Montluçon - ville,12:14:30,12:09:30,09:58:00,5,Conditions météorologiques
21020,10-03-2023,Bellegarde-sur-Valserine Gare,Lyon-Part-Dieu - Gare Routière,11:55:00,11:50:00,10:10:00,5,Embouteillage


On exporte le dataframe en fichier _CSV_

In [41]:
df_retard.to_csv('Delay.csv', sep=',', index=False, header=True)